In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.set_defaults='kaggle'
pd.set_option('display.max_columns', None)

Certainly! Here's an explanation of the features in the backorder prediction dataset:

1. **sku**: Stock Keeping Unit, a unique identifier for a specific product or item in inventory.

2. **national_inv**: The current inventory level of the product at a national level.

3. **lead_time**: The average lead time (in days) it takes to restock the product once it's ordered.

4. **in_transit_qty**: The quantity of the product that is currently in transit, i.e., on its way to the inventory.

5. **forecast_3_month**, **forecast_6_month**, **forecast_9_month**: Forecasted demand for the product over the next 3, 6, and 9 months, respectively.

6. **sales_1_month**, **sales_3_month**, **sales_6_month**, **sales_9_month**: Actual sales quantity for the product over the past 1, 3, 6, and 9 months, respectively.

7. **min_bank**: The minimum acceptable inventory level for the product, ensuring availability during lead time.

8. **potential_issue**: A binary indicator (Yes/No) that signifies whether the product has had a potential issue in the past.

9. **pieces_past_due**: The number of pieces of the product that were past due for delivery.

10. **perf_6_month_avg**, **perf_12_month_avg**: Performance metrics indicating the average fulfillment rate of the product over the past 6 and 12 months, respectively.

11. **local_bo_qty**: The quantity of the product currently on backorder at a local level.

12. **deck_risk**: A binary indicator (Yes/No) that assesses the risk associated with keeping the product on deck.

13. **oe_constraint**: A binary indicator (Yes/No) indicating whether there's an operational engineering constraint on the product.

14. **ppap_risk**: A binary indicator (Yes/No) indicating whether the product has a Production Part Approval Process (PPAP) risk.

15. **stop_auto_buy**: A binary indicator (Yes/No) indicating whether automatic buying of the product is stopped.

16. **rev_stop**: A binary indicator (Yes/No) indicating whether revenue recognition is stopped for the product.

17. **went_on_backorder**: The target variable, a binary indicator (Yes/No) indicating whether the product went on backorder or not.

These features provide information about the product's inventory levels, demand forecasts, sales history, performance metrics, risk factors, and potential issues. They play a crucial role in building predictive models to identify the likelihood of a product going on backorder, which is valuable for optimizing inventory management and ensuring customer satisfaction.

In [18]:
train_df=pd.read_csv('/kaggle/input/backorder/Kaggle_Training_Dataset_v2.csv')
test_df=pd.read_csv('/kaggle/input/backorder/Kaggle_Test_Dataset_v2.csv')


/tmp/ipykernel_33/185334582.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.

/tmp/ipykernel_33/185334582.py:2: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [19]:
train_df.shape,test_df.shape

((1687861, 23), (242076, 23))

In [20]:
display(train_df.sample(5),test_df.sample(5))

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
124844,1236361,30.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,9.0,1.0,No,0.0,0.85,0.51,0.0,No,No,No,Yes,No,No
1671954,3271217,3.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.10,0.11,0.0,No,No,No,Yes,No,No
40906,1152395,5.0,12.0,0.0,10.0,10.0,15.0,1.0,3.0,9.0,12.0,0.0,No,0.0,0.05,0.24,0.0,No,No,No,Yes,No,No
671954,2019939,227.0,8.0,1.0,0.0,0.0,0.0,4.0,14.0,30.0,52.0,7.0,No,0.0,0.98,0.92,0.0,No,No,No,Yes,No,No
691329,2039319,20.0,6.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,No,0.0,1.00,0.98,0.0,No,No,No,Yes,No,No


,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
134867,3416190,374.0,2.0,0.0,270.0,540.0,630.0,47.0,183.0,507.0,752.0,61.0,No,0.0,0.97,0.97,0.0,No,No,No,Yes,No,No
39468,3317413,1.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,No,0.0,0.83,0.92,0.0,No,No,No,Yes,No,No
204897,3488619,19.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.98,0.90,0.0,No,No,No,Yes,No,No
194575,3477958,3.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,0.70,0.66,0.0,No,No,No,Yes,No,No
32609,3310288,10.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,No,0.0,0.54,0.56,0.0,Yes,No,No,Yes,No,No


In [21]:
display(
    train_df.info(),
    test_df.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 23 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   sku                1687861 non-null  object 
 1   national_inv       1687860 non-null  float64
 2   lead_time          1586967 non-null  float64
 3   in_transit_qty     1687860 non-null  float64
 4   forecast_3_month   1687860 non-null  float64
 5   forecast_6_month   1687860 non-null  float64
 6   forecast_9_month   1687860 non-null  float64
 7   sales_1_month      1687860 non-null  float64
 8   sales_3_month      1687860 non-null  float64
 9   sales_6_month      1687860 non-null  float64
 10  sales_9_month      1687860 non-null  float64
 11  min_bank           1687860 non-null  float64
 12  potential_issue    1687860 non-null  object 
 13  pieces_past_due    1687860 non-null  float64
 14  perf_6_month_avg   1687860 non-null  float64
 15  perf_12_month_avg  1687860 non-n

None

None

In [22]:
display(
    train_df.describe().T,
    test_df.describe().T
)

,count,mean,std,min,25%,50%,75%,max
national_inv,1687860.0,496.111782,29615.233831,-27256.0,4.00,15.00,80.00,12334404.0
lead_time,1586967.0,7.872267,7.056024,0.0,4.00,8.00,9.00,52.0
in_transit_qty,1687860.0,44.052022,1342.741731,0.0,0.00,0.00,0.00,489408.0
forecast_3_month,1687860.0,178.119284,5026.553102,0.0,0.00,0.00,4.00,1427612.0
forecast_6_month,1687860.0,344.986664,9795.151861,0.0,0.00,0.00,12.00,2461360.0
forecast_9_month,1687860.0,506.364431,14378.923562,0.0,0.00,0.00,20.00,3777304.0
sales_1_month,1687860.0,55.926069,1928.195879,0.0,0.00,0.00,4.00,741774.0
sales_3_month,1687860.0,175.025930,5192.377625,0.0,0.00,1.00,15.00,1105478.0
sales_6_month,1687860.0,341.728839,9613.167104,0.0,0.00,2.00,31.00,2146625.0
sales_9_month,1687860.0,525.269701,14838.613523,0.0,0.00,4.00,47.00,3205172.0


,count,mean,std,min,25%,50%,75%,max
national_inv,242075.0,499.751028,29280.390793,-25414.0,4.00,15.00,81.00,12145792.0
lead_time,227351.0,7.923018,7.041410,0.0,4.00,8.00,9.00,52.0
in_transit_qty,242075.0,36.178213,898.673127,0.0,0.00,0.00,0.00,265272.0
forecast_3_month,242075.0,181.472345,5648.874620,0.0,0.00,0.00,4.00,1510592.0
forecast_6_month,242075.0,348.807304,10081.797119,0.0,0.00,0.00,12.00,2157024.0
forecast_9_month,242075.0,508.296301,14109.723787,0.0,0.00,0.00,20.00,3162260.0
sales_1_month,242075.0,51.478195,1544.678350,0.0,0.00,0.00,4.00,349620.0
sales_3_month,242075.0,172.139316,5164.243624,0.0,0.00,1.00,14.00,1099852.0
sales_6_month,242075.0,340.425414,9386.523492,0.0,0.00,2.00,30.00,2103389.0
sales_9_month,242075.0,511.775446,13976.702192,0.0,0.00,4.00,46.00,3195211.0


In [23]:
display(
    train_df.describe(exclude= np.number).T,
    test_df.describe(exclude= np.number).T
)

,count,unique,top,freq
sku,1687861,1687861,1026827,1
potential_issue,1687860,2,No,1686953
deck_risk,1687860,2,No,1300377
oe_constraint,1687860,2,No,1687615
ppap_risk,1687860,2,No,1484026
stop_auto_buy,1687860,2,Yes,1626774
rev_stop,1687860,2,No,1687129
went_on_backorder,1687860,2,No,1676567


,count,unique,top,freq
sku,242076,242076,3285085,1
potential_issue,242075,2,No,241993
deck_risk,242075,2,No,194105
oe_constraint,242075,2,No,242028
ppap_risk,242075,2,No,213357
stop_auto_buy,242075,2,Yes,232617
rev_stop,242075,2,No,241967
went_on_backorder,242075,2,No,239387


In [24]:
display(
    train_df.isnull().mean()*100,
    test_df.isnull().mean()*100
)

sku                  0.000000
national_inv         0.000059
lead_time            5.977625
in_transit_qty       0.000059
forecast_3_month     0.000059
forecast_6_month     0.000059
forecast_9_month     0.000059
sales_1_month        0.000059
sales_3_month        0.000059
sales_6_month        0.000059
sales_9_month        0.000059
min_bank             0.000059
potential_issue      0.000059
pieces_past_due      0.000059
perf_6_month_avg     0.000059
perf_12_month_avg    0.000059
local_bo_qty         0.000059
deck_risk            0.000059
oe_constraint        0.000059
ppap_risk            0.000059
stop_auto_buy        0.000059
rev_stop             0.000059
went_on_backorder    0.000059
dtype: float64

sku                  0.000000
national_inv         0.000413
lead_time            6.082800
in_transit_qty       0.000413
forecast_3_month     0.000413
forecast_6_month     0.000413
forecast_9_month     0.000413
sales_1_month        0.000413
sales_3_month        0.000413
sales_6_month        0.000413
sales_9_month        0.000413
min_bank             0.000413
potential_issue      0.000413
pieces_past_due      0.000413
perf_6_month_avg     0.000413
perf_12_month_avg    0.000413
local_bo_qty         0.000413
deck_risk            0.000413
oe_constraint        0.000413
ppap_risk            0.000413
stop_auto_buy        0.000413
rev_stop             0.000413
went_on_backorder    0.000413
dtype: float64

In [25]:
x=train_df.corr()
y=test_df.corr()
display(
    px.imshow(x,text_auto=True),
    px.imshow(y,text_auto=True)
)

/tmp/ipykernel_33/4249510676.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

/tmp/ipykernel_33/4249510676.py:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



# let's Fight With NULL Values

i have found that row 1687860 contains null in all features so lets remove it

In [27]:
train_df[train_df['national_inv'].isnull()]

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1687860,(1687860 rows),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
train_df.drop(index=1687860, inplace=True)

In [29]:
train_df['lead_time'].isnull().mean()*100

5.9775692296754475

in lead_time featuer most values are near to median so replacing null vales with median will br good 

In [41]:
def median_replace(df,col):
    print(f"null values in{col} : {df[col].isnull().sum()}")
    df[col]=df[col].fillna(df[col].median())
    print(f"after removing null values : {df[col].isnull().sum()}")

In [42]:
median_replace(train_df,'lead_time')

null values inlead_time : 100893
after removing null values : 0


In [45]:
train_df.isnull().sum()

sku                  0
national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

Now we have zero null values in our dataset